In [ ]:
from urllib import parse, request
import hashlib
import datetime
from time import strftime, gmtime
import time
import pandas as pd
from io import StringIO
import pyodbc
from sqlalchemy.engine import URL
from urllib import error
from config import *

api_version = 3.0

# Initial Configuration
* Set time frame you want to pull. 
    * Commissions are locked on the 20th of the following month.
      (E.g. If today were 9/19, we would want to audit transactions from 8/1 - 8/31.)
* Set the brand initials to auto assign the appropriate api keys. 

In [ ]:
# Set date parameters.
date_start = datetime.date(2023, 12, 1)
date_end = datetime.date(2023, 12,31)

# Set brand.
brand = "ST"

# Assigning private_key based on db value.
if brand == "CT":
    store_id = ct_store
    api_token = ct_api_token
    secret_key = ct_api_secret_key
    connection_string = ct_connection_string
elif brand == "KP":
    store_id = kp_store
    api_token = kp_api_token
    secret_key = kp_api_secret_key
    connection_string = kp_connection_string
elif brand == "ST":
    store_id = st_store
    api_token = st_api_token
    secret_key = st_api_secret_key
    connection_string = st_connection_string
elif brand == "WC":
    store_id = wc_store
    api_token = wc_api_token
    secret_key = wc_api_secret_key
    connection_string = wc_connection_string

In [ ]:
# Set parameters.
data = parse.urlencode({'merchantId': store_id, 'token': api_token,
'version': api_version, 'action': 'transactiondetail', 'sortCol' : 'transid',
'dateStart' : date_start, 'dateEnd' : date_end, 'format': 'pipe'})

my_timestamp = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())

# API Auth.
sig = api_token + ':' + my_timestamp + ':' + 'transactiondetail' + ':' + secret_key
sig_hash = hashlib.sha256(sig.encode('utf-8')).hexdigest()
my_headers = {'x-ShareASale-Date': my_timestamp, 'x-ShareASale-Authentication': sig_hash}

# Call.
call = request.Request('https://shareasale.com/w.cfm?%s' % data, headers=my_headers)

try:
    response = request.urlopen(call).read()
except Exception as inst:
    print(inst)
    
# Decode output into df.
output = response.decode("utf-8")

# Push into df.
df = pd.read_csv(StringIO(output), sep='|', engine= 'python')

# Check row counts, and if date parameters are in line.     
print(f"Transactions Pulled: {df.shape[0]}, \nMinimum date: {min(df['transdate'])}, \nMaximum date: {max(df['transdate'])}")

In [ ]:
# Now we need to specify that we're looking for transactions that haven't been voided or locked for payment.
df = df.query("voided.isna() and locked.isna()")[['orderNumber', 'transdate', 'transamount', 'commission']]
print(f"Transactions to audit: {df.shape[0]}")

## Part One: Subtotal Corrections

* In their contracts, affiliates aren't to be paid on digital items such as patterns or gift cards. (Licensing reasons, small margins, etc.)
* We're going to query our servers looking for order id's that contain any digital items.
If they do, we subtract the gross price from the subtotal, and create a data frame with the order ids and new subtotal for later processing.
    * Note: If a subtotal is reduced to 0, it is voided, so we don't have to worry about error handling.

In [ ]:
# Init connection.
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

#ODBC Engine
from sqlalchemy import create_engine
engine = create_engine(connection_url)

# ODBC call.
cursor = engine.connect()

In [ ]:
SQL = (
"""
SELECT
    A.orderdate,
    A.orderno,
    A.weborderid,
    A.netamt,
    CAST(A.netamt - B.adjustedgross AS DECIMAL(6,2)) AS adjustednetamt
FROM """+brand+""".reporting.Transactions A
INNER JOIN ( 
    SELECT 
        A.orderno,
        SUM(A.extendedprice) AS adjustedgross
    FROM """+brand+""".reporting.TransactionLine A
    WHERE A.productclass in ('Books - Digital', 'Patterns - Digital')
    OR A.itemid = 'GIFT'
    GROUP BY A.orderno
) AS B ON A.orderno = B.orderno
WHERE A.weborderid IN {}""".format(tuple(str(x) for x in df['orderNumber'])) 
)

# ODBC Call
net_adjustment = pd.read_sql(SQL, engine)

# Subtract adjusted from orders subtotal. Take percentage to find commission savings.
net_adjustment['savings'] = (net_adjustment['netamt'] - net_adjustment['adjustednetamt']) * 0.1

if not net_adjustment.empty:
    #Print summary.
    print(f"Transactions: {net_adjustment.shape[0]}, \nSavings Found: ${net_adjustment['savings'].sum().round(2)} ")
else:
    print("No transactions meet the parameters.")

## Part Two: Canceled Orders

* Look for orders that have been canceled, involved in a chargeback, etc.

In [ ]:
SQL = (
"""
SELECT
    A.orderdate,
    A.orderno,
    A.weborderid,
    A.netamt
FROM """+brand+""".reporting.Transactions A
WHERE A.orderstatus IN ('Closed', 'Cancelled')
AND A.weborderid IN {}""".format(tuple(str(x) for x in df['orderNumber'])) 
)

# ODBC Call
canceled = pd.read_sql(SQL, engine)

# Take 10% of the net amount to find savings.
canceled['savings'] = (canceled['netamt']) * 0.1

if not canceled.empty:
    #Print summary.
    print(f"Transactions: {canceled.shape[0]}, \nSavings Found: ${canceled['savings'].sum().round(2)} ")
else:
    print("No transactions meet the parameters.")

## Part Three: Void/Edit Orders

* Recrusively go through through the newly created dataframes with the API calls below.
    * editAPI needs date, ordernumber, and newamount.
    * voidAPI needs date, and ordernumber.

In [ ]:
# Changing the total of the order.
def editAPI(ordernumber, date, newamount):
    data = parse.urlencode({'merchantId': store_id, 'token': api_token, 'version': api_version, 'action': 'edit', 'date': date, 'ordernumber': ordernumber, 'newamount': newamount, 'newcomment': 'Edit: Remov ineligible items from subtotal.'})
    sig = api_token + ':' + my_timestamp + ':' + 'edit' + ':' + secret_key
    sig_hash = hashlib.sha256(sig.encode('utf-8')).hexdigest()
    my_headers = {'x-ShareASale-Date': my_timestamp, 'x-ShareASale-Authentication': sig_hash}  
    try:
        call = request.Request('https://shareasale.com/w.cfm?%s' % data, headers=my_headers)
        response = request.urlopen(call)
        response_data = response.read()
        decoded_response = response_data.decode('utf-8')
        clean_response = decoded_response.replace('\r', '').replace('\n', '').replace('\t', '').strip()
        print(f"Success for ordernumber {ordernumber}: {clean_response}")
    except error.HTTPError as e:
        print(f"HTTP Error for ordernumber {ordernumber}: {e.code} {e.read()}")
    except Exception as e:
        print(f"Error for ordernumber {ordernumber}: {e}")
    return

# Change order total.
for index, row in net_adjustment.iterrows():
    editAPI(row['weborderid'], row['orderdate'], row['adjustednetamt'])
    time.sleep(1) # Throws errors if too many pass to quickly.

In [ ]:
#Canceling the order.
def voidAPI(ordernumber, date):
    data = parse.urlencode({'merchantId': store_id, 'token': api_token, 'version': api_version, 'action': 'void', 'ordernumber': ordernumber, 'date': date, 'reason': 'Order was cancelled.'})
    sig = api_token + ':' + my_timestamp + ':' + 'void' + ':' + secret_key
    sig_hash = hashlib.sha256(sig.encode('utf-8')).hexdigest()
    my_headers = {'x-ShareASale-Date': my_timestamp, 'x-ShareASale-Authentication': sig_hash}
    try:
        call = request.Request('https://shareasale.com/w.cfm?%s' % data, headers=my_headers)
        response = request.urlopen(call)
        response_data = response.read()
        decoded_response = response_data.decode('utf-8')
        clean_response = decoded_response.replace('\r', '').replace('\n', '').replace('\t', '').strip()
        print(f"Success for ordernumber {ordernumber}: {clean_response}")
    except error.HTTPError as e:
        print(f"HTTP Error for ordernumber {ordernumber}: {e.code} {e.read()}")
    except Exception as e:
        print(f"Error for ordernumber {ordernumber}: {e}")
    return

# Void canceled orders.
for index, row in canceled.iterrows():
    voidAPI(row['weborderid'], row['orderdate'])
    time.sleep(1) # Throws errors if too many pass to quickly.